In [1]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame 

import matplotlib.pyplot as plt 
import seaborn as sns
import itertools
%matplotlib inline
from matplotlib.pyplot import figure
import matplotlib as mpl
import matplotlib.font_manager as fm
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'Malgun Gothic' 
plt.rcParams["font.size"] = 10
%matplotlib inline 


from datetime import datetime, timedelta 
from statsmodels.tsa.arima_model import ARIMA 

from sklearn.preprocessing import StandardScaler
import os

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [6]:
# test = pd.read_csv("/kaggle/input/dataset/test.csv", sep = ",")
# train = pd.read_csv("/kaggle/input/dataset/train.csv", sep =",")
# submission = pd.read_csv("/kaggle/input/dataset/submission.csv")

test = pd.read_csv("data/test.csv", sep = ",")
train = pd.read_csv("data/train.csv", sep =",")
submission = pd.read_csv("data/submission.csv")

## Clustering by Ratio(미터ID마다 시간별 값/lag(1일, 7일))

In [7]:
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import fcluster

In [9]:
train2 = train.copy()
train3 = pd.melt(train2, id_vars=['Time'], value_vars=train2.columns[1:1292])

In [ ]:
train4 = train3.groupby(['Time','variable'])['value'].agg({'value':'mean'}).reset_index()

In [ ]:
train5 = train4.copy()
train5['lag1'] = train5.groupby(['variable'])['value'].shift(1)
train5['lag7'] = train5.groupby(['variable'])['value'].shift(7)
train5 = train5.fillna(0)
train5['ratio1'] = train5['value']/(train5['lag1']+1)
train5['ratio7'] = train5['value']/(train5['lag7']+1)
train5.head(2)

In [ ]:
train6 = train5.groupby(['variable']).agg({'ratio1':'mean','ratio7':'mean'}).reset_index()
featureTr = train6.copy()
plt.scatter(featureTr['ratio1'], featureTr['ratio7'])

In [ ]:
featureTr['id'] = [int(s[1:]) for s in featureTr['variable']]
featureTr2 = featureTr.iloc[:, 1:]
featureTr2

In [ ]:
Z = linkage(featureTr2,'ward')

plt.figure(figsize=(50,20))
dendrogram(Z,
           leaf_rotation=100,
           leaf_font_size=1,
)
plt.show()

In [ ]:
k=3
featureTr3 = featureTr.copy()
featureTr3 = featureTr3.drop(columns='id')
predict = fcluster(Z, k, criterion = 'maxclust')
featureTr3['cluster']=predict
train7 = pd.merge(train5, featureTr3, on ='variable',how='left')

c_1T = train7[train7['cluster']==1]
c_2T = train7[train7['cluster']==2]
c_3T = train7[train7['cluster']==3]

In [ ]:
cnt_srs_1T = c_1T.groupby('Time')['value'].mean()
cnt_srs_2T = c_2T.groupby('Time')['value'].mean()
cnt_srs_3T = c_3T.groupby('Time')['value'].mean()

In [ ]:
import plotly.graph_objs as go
def scatter_plot(cnt_srs, color, showlegend1=False, name1 = ""):
    trace = go.Scatter(
        x=cnt_srs.index[::-1],
        y=cnt_srs.values[::-1],
        name = name1,
        showlegend=showlegend1,
        marker=dict(
            color=color,
        ),
    )
    return trace

In [ ]:
trace_1T = scatter_plot(cnt_srs_1T, 'yellow', True ,'Cluster_1')
trace_2T = scatter_plot(cnt_srs_2T, 'green', True ,'Cluster_2')
trace_3T = scatter_plot(cnt_srs_3T, 'red', True ,'Cluster_3')

In [ ]:
import plotly.offline as py
layout = go.Layout(
    height=500,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    title='Train Clustering_3'
)

data = [trace_1T,trace_2T, trace_3T]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

In [ ]:
test2 = test.copy()

In [ ]:
test2.reset_index(level=0, inplace=True)
v = test2.columns[1:12]
test3 = pd.melt(test2, id_vars=['Time'], value_vars=test2.columns[1:1292])

In [ ]:
test4 = test3.groupby(['Time','variable'])['value'].agg({'value':'mean'}).reset_index()

In [ ]:
test5 = test4.copy()
test5['lag1'] = test5.groupby(['variable'])['value'].shift(1)
test5['lag7'] = test5.groupby(['variable'])['value'].shift(7)
test5 = test5.fillna(0)
test5['ratio1'] = test5['value']/(test5['lag1']+1)
test5['ratio7'] = test5['value']/(test5['lag7']+1)
test5.head(2)

In [ ]:
test6 = test5.groupby(['variable']).agg({'ratio1':'mean','ratio7':'mean'}).reset_index()
feature = test6
plt.scatter(feature['ratio1'], feature['ratio7'])

In [ ]:
feature['id'] = [int(s[1:]) for s in feature['variable']]
feature2 = feature.iloc[:, 1:]
feature2

In [ ]:
Z = linkage(feature2,'ward')

plt.figure(figsize=(50,20))
dendrogram(Z,
           leaf_rotation=100,
           leaf_font_size=1,
)
plt.show()

In [ ]:
feature3 = feature.copy()
feature3 = feature3.drop(columns='id')

In [ ]:
k=3

predict = fcluster(Z, k, criterion = 'maxclust')
feature3['cluster']=predict
test7 = pd.merge(test5, feature3, on ='variable',how='left')

c_1 = test7[test7['cluster']==1]
c_2 = test7[test7['cluster']==2]
c_3 = test7[test7['cluster']==3]
# c_4 = test7[test7['cluster']==4]

In [ ]:
cnt_srs_1 = c_1.groupby('Time')['value'].mean()
cnt_srs_2 = c_2.groupby('Time')['value'].mean()
cnt_srs_3 = c_3.groupby('Time')['value'].mean()
# cnt_srs_4 = c_4.groupby('Time')['value'].mean()

In [ ]:
import plotly.graph_objs as go
def scatter_plot(cnt_srs, color, showlegend1=False, name1 = ""):
    trace = go.Scatter(
        x=cnt_srs.index[::-1],
        y=cnt_srs.values[::-1],
        name = name1,
        showlegend=showlegend1,
        marker=dict(
            color=color,
        ),
    )
    return trace

In [ ]:
trace_1 = scatter_plot(cnt_srs_1, 'yellow', True ,'Cluster_1')
trace_2 = scatter_plot(cnt_srs_2, 'green', True ,'Cluster_2')
trace_3 = scatter_plot(cnt_srs_3, 'red', True ,'Cluster_3')
# trace_4 = scatter_plot(cnt_srs_3, 'grey', True ,'Cluster_3')

In [ ]:
import plotly.offline as py
layout = go.Layout(
    height=500,
    width=1000,
    #paper_bgcolor='rgb(233,233,233)',
    title='Test Clustering_2'
)

data = [trace_1,trace_2, trace_3]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

In [ ]:
test7.groupby('cluster')['variable'].nunique()

In [ ]:
# TEST에서 각각의 클러스터에 해당하는 ID 추출
c1_testID = np.unique(test7[test7.cluster==1]['variable'])
c2_testID = np.unique(test7[test7.cluster==2]['variable'])
c3_testID = np.unique(test7[test7.cluster==3]['variable'])

In [ ]:
print(c1_testID,'\n',c2_testID,'\n', c3_testID)

## NA 채우기
- 요일+시간가중치+Dacon적용, 누적값 분산한 ch_Test, ch_Train을 각각 50% 이상의 데이터가 존재하는 시간대에서 cut
- Train (X118) 데이터 값이 1개여서 추가 drop
- Arima를 이용해 단순 예측한 24hr을 시간에 맞춰 대입

In [ ]:
changed_train = pd.read_csv("data/train_DHWeight_onDacon.csv", index_col=0, sep=',')
changed_test = pd.read_csv("data/test_DHWeight_onDacon.csv", index_col=0, sep=',')

In [ ]:
changed_test.shape

In [ ]:
changed_test.index = pd.to_datetime(changed_test.index) 
changed_train.index = pd.to_datetime(changed_train.index) 

In [ ]:
idx = changed_test.count(axis = 'columns') >= 200*0.5
ch_te_TimeCut = changed_test[changed_test[idx].index[0]:]

In [ ]:
arima_test = pd.read_csv("data/TEST_DHWonDacon_24시간예측.csv",index_col=0, sep=',')
arima_train = pd.read_csv("data/TRAIN_DHWonDacon_24시간예측.csv",index_col=0, sep=',')

In [ ]:
def fillNA(df1, df2):
    df = pd.DataFrame()
    n=0
    for i in df1.columns:
        n += 1
        li = df1[df1[i].isnull()].index
        tmp = pd.DataFrame(df1[i]) 
        print("%d. current column: %s, NaN count: %d" %(n, i, len(li)))
        for time in li:
            h = time.hour
            tmp.loc[time] = df2.iloc[h][i]
        df = pd.concat([df,tmp], axis=1) 
    return df

In [ ]:
TE_TimeCut_NAfilled = fillNA(ch_te_TimeCut, arima_test)
TE_TimeCut_NAfilled.shape

In [ ]:
ch_te_TimeCut.iloc[:,ch_te_TimeCut.isnull().sum().values == 5577].columns

In [ ]:
idx = changed_train.count(axis = 'columns') >= 1292*0.5
ch_tr_TimeCut = changed_train[changed_train[idx].index[0]:]
ch_tr_TimeCut

In [ ]:
# drop a columns X118 since it is Empty in Arima_Train df
ch_tr_TimeCut = ch_tr_TimeCut.drop(columns = 'X118')

In [ ]:
TR_TimeCut_NAfilled = fillNA(ch_tr_TimeCut, arima_train)
TR_TimeCut_NAfilled.shape

In [ ]:
TE_TimeCut_NAfilled.to_csv("data/TEST_DHweightDaconArima_NAFilled.csv")

In [ ]:
TR_TimeCut_NAfilled.to_csv("data/TRAIN_DHweightDaconArima_NAFilled.csv")

## Weather 추가변수
- 나중에 모델 돌릴때마다 df에 추가하는 것

In [9]:
# hourly WEATHER data
w_hr = pd.read_csv("/kaggle/input/weather/hourly.csv",encoding='euc-kr', sep=',')
w_hr2 = w_hr.loc[:,['Time', 'Temp','Humidity', 'WindSpeed']]
w_hr2['Humidity']=w_hr2.loc[:,'Humidity'].fillna(0)
w_hr2.head(2)

,Time,Temp,Humidity,WindSpeed
0,2016.7.20 0:00,25.7,0.0,1.3
1,2016.7.20 1:00,25.6,0.0,1.4


In [10]:
w_hr2['Time'] = pd.to_datetime(w_hr2.Time)
w_hr2 = w_hr2.set_index('Time')
w_hr2.head(2)

,Temp,Humidity,WindSpeed
Time,,,
2016-07-20 00:00:00,25.7,0.0,1.3
2016-07-20 01:00:00,25.6,0.0,1.4


In [ ]:
# daily WEATHER data
# w_daily = pd.read_csv("data/weather/인천_일별_기상자료(16-18)_축소.csv",encoding='euc-kr', sep=',')
# w_daily2 = w_daily.iloc[:,1:5]
# old = w_daily2.columns
# new = ['Time', 'AvgT', 'MinT', 'MaxT']
# w_daily2 = w_daily2.rename(columns=dict(zip(old, new)))
# w_daily2.head(2)

In [ ]:
# w_daily2['Time'] = pd.to_datetime(w_daily2.Time)
# w_daily2 = w_daily2.set_index('Time')
# w_daily2.head(2)

## 파생 및 dummy변수

In [11]:
test = pd.read_csv("/kaggle/input/dataset/TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')
train = pd.read_csv("/kaggle/input/dataset/TRAIN_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')

In [12]:
test.head(1)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.29667


In [13]:
test.index = pd.to_datetime(test.index)

In [16]:
ori_test = pd.read_csv("/kaggle/input/dataset/test.csv", index_col = 0)
ori_test.head(1)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017.7.1 0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.275,0.021,NaN


In [17]:
def dateTimeVar(df):
    df1 = pd.DataFrame(df.index)
    df1.index = pd.to_datetime(df1.iloc[:,0])
    df1 = df1.drop(columns=df1.columns)
    # datetime을 사용한 파생변수
    # 연도, 월, 일, 요일, 시간
    df1['Mth']= df1.index.month
    df1['Date']=df1.index.day
    df1['Day']=df1.index.weekday
    df1['hour']=df1.index.hour
    
    # Season 1: 봄 ~ 4: 겨울 
    df1['season'] = df1.Mth.apply(lambda x: 4 if x in [12,1,2] else 
                1 if x in [3,4,5] else 2 if x in [6,7,8]
               else 3)
    # dayClass 1: 월, 2: 화~금, 3: 토, 일 
    # 참조 from 실시간 수요예측 기법 개발 및 온라인 수요예측 방안 연구_KPX 한국전력거래소)
    df1['dayClass'] = df1.Day.apply(lambda x: 1 if x ==0 else 
                2 if x in [1,2,3,4] else 3)
    return df1

In [18]:
TE_time_vars = dateTimeVar(test)

In [19]:
TE_time_vars.head(1)

,Mth,Date,Day,hour,season,dayClass
Time,,,,,,
2017-08-29 10:00:00,8,29,1,10,2,2


**파생변수와 TS관계 (클러스터별 random sample 3개씩)**

In [ ]:
from random import seed,randint

In [ ]:
test_timevars = pd.concat([test, TE_time_vars], axis=1)

In [ ]:
rd_test1 = [] ; rd_test2 = [] ; rd_test3 = []
# seed(2.83)
# for _ in range(3):
#     rd_test1.append(c1_testID[randint(0,len(c1_testID))])
# for _ in range(3):
#     rd_test2.append(c2_testID[randint(0,len(c2_testID))])
# for _ in range(3):
#     rd_test3.append(c3_testID[randint(0,len(c3_testID))])
rd_test1 = ['X42', 'X111', 'X20']
rd_test2 = ['X216', 'X248', 'X245']
rd_test3 = ['X422', 'X398', 'X399']

In [ ]:
test_timevars.groupby('Mth')[rd_test1].mean().plot()

In [ ]:
test_timevars.groupby('Mth')[rd_test2].mean().plot()

In [ ]:
test_timevars.groupby('Mth')[rd_test3].mean().plot()

# 그래프그래프그으으으래래래ㅐㄹ프 그려라 

In [ ]:
# plt.rcParams.update({'font.size': 15})
# fig = plt.figure(figsize=(60,14))
# clusters = [rd_test1, rd_test2, rd_test3]

# for i in range(1,3):
#     for k in range(1,4):
#         ax = fig.add_subplot(i,3,k)
#         ax.spines['top'].set_visible(False) ; ax.spines['right'].set_visible(False)
#         if i==1:
#             ax.set_title('월별 전력사용량_Cluster'+str(i))
#             ax.plot(test1.groupby('Mth')[clusters[k-1]].mean(), linewidth=4.0)  
#         else:
#             ax.set_title('계절별 전력사용량_Cluster'+str(i))
#             ax.plot(test1.groupby('season')[clusters[k-1]].mean())

#         ax.legend(clusters[k-1])

In [ ]:
# test1.groupby('Day')[clusters[0]].mean().plot()
# test1.groupby('dayClass')[clusters[0]].mean().plot.bar()

In [ ]:
# test1.groupby('Day')[clusters[1]].mean().plot()
# test1.groupby('dayClass')[clusters[1]].mean().plot.bar()

In [ ]:
# test1.groupby('Day')[clusters[2]].mean().plot()
# test1.groupby('dayClass')[clusters[2]].mean().plot.bar()

## Make DUMMIES

In [20]:
test.head(2)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.29667
2017-08-29 11:00:00,1.458903,0.398,0.293,0.358,0.350,0.232,0.253,0.260,0.168,0.084,...,0.163,0.086,0.288,0.342010,1.150483,0.196,0.343,1.109,0.225,0.29667


In [21]:
TE_time_vars.head(2)

,Mth,Date,Day,hour,season,dayClass
Time,,,,,,
2017-08-29 10:00:00,8,29,1,10,2,2
2017-08-29 11:00:00,8,29,1,11,2,2


In [22]:
def dummies(df, dum_lst):
    df1 = df.copy()
    # dum_lst에 들어있는 각각의 변수마다 
    # tmp = get_dummies, 해당 original 변수는 drop
    # tmp와 변수가 drop된 df를 join 후 리턴
    for v in dum_lst:
        tmp = pd.get_dummies(df1[v], prefix=v)
        df1 = df1.drop(columns=v).join(tmp)
    return df1

In [23]:
TE_timeV_dum = dummies(TE_time_vars, ['season', 'dayClass'])

In [24]:
TE_timeV_dum.head()

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3
Time,,,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0
2017-08-29 11:00:00,8,29,1,11,0,1,0,0,0,1,0
2017-08-29 12:00:00,8,29,1,12,0,1,0,0,0,1,0
2017-08-29 13:00:00,8,29,1,13,0,1,0,0,0,1,0
2017-08-29 14:00:00,8,29,1,14,0,1,0,0,0,1,0


In [25]:
TE_timeV_dum_wthr= pd.concat([TE_timeV_dum, w_hr2], axis=1,join='inner')

In [27]:
TE_timeV_dum_wthr.head(2)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed
Time,,,,,,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0,19.3,0.0,5.0
2017-08-29 11:00:00,8,29,1,11,0,1,0,0,0,1,0,19.7,0.0,4.7


In [28]:
## No DUMMIES only 시간 파생 + 날씨!! (LGBM try용)

TE_timeV_wthr= pd.concat([TE_time_vars, w_hr2], axis=1,join='inner')
TE_timeV_wthr.head(2)

,Mth,Date,Day,hour,season,dayClass,Temp,Humidity,WindSpeed
Time,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,2,2,19.3,0.0,5.0
2017-08-29 11:00:00,8,29,1,11,2,2,19.7,0.0,4.7


# Modeling

* test : TEST셋의 순수한 meterIDs Time series (NA값 채워짐) 
* TE_timeV_dum: TEST셋의 시간파생변수 + dummy variables 

In [29]:
test.head(1)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.29667


In [ ]:
TE_timeV_dum_wthr.head(1)

In [57]:
data = pd.concat([pd.DataFrame(test.X303), TE_timeV_dum_wthr], axis=1)
data = data.rename(columns={'X303':'y'})

for i in [24, 168]:
    data['lag_{}'.format(i)] = data.y.shift(i)

In [58]:
data.shape

(7334, 17)

In [59]:
y = data.dropna().y
X = data.dropna().drop(['y'], axis=1)

In [ ]:
y_2 = data['2018-04-18 00:00:00':].dropna().y
X_2 = data['2018-04-18 00:00:00':].dropna().drop(['y'], axis=1)

In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit

# for time-series cross-validation set 5 folds 
tscv = TimeSeriesSplit(n_splits=5)

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def timeseries_train_test_split(X, y, test_size):
    """
        Perform train-test split with respect to time series structure
    """
    # get the index after which test set starts
    test_index = int(len(X)*(1-test_size))

    X_train = X.iloc[:test_index]
    y_train = y.iloc[:test_index]
    X_test = X.iloc[test_index:]
    y_test = y.iloc[test_index:]
    
    return X_train, X_test, y_train, y_test

In [51]:
# def plotModelResults(model, X_train, X_test, plot_intervals=False, plot_anomalies=False, scale=1.96):
#     """
#         Plots modelled vs fact values, prediction intervals and anomalies
    
#     """
#     prediction = model.predict(X_test)
    
#     plt.figure(figsize=(15, 7))
#     plt.plot(prediction, "g", label="prediction", linewidth=2.0)
#     plt.plot(y_test.values, label="actual", linewidth=2.0)
    
#     if plot_intervals:
#         cv = cross_val_score(model, X_train, y_train, 
#                                     cv=tscv, 
#                                     scoring="neg_mean_squared_error")
#         #mae = cv.mean() * (-1)
#         deviation = np.sqrt(cv.std())
        
#         lower = prediction - (scale * deviation)
#         upper = prediction + (scale * deviation)
        
#         plt.plot(lower, "r--", label="upper bond / lower bond", alpha=0.5)
#         plt.plot(upper, "r--", alpha=0.5)
        
#         if plot_anomalies:
#             anomalies = np.array([np.NaN]*len(y_test))
#             anomalies[y_test<lower] = y_test[y_test<lower]
#             anomalies[y_test>upper] = y_test[y_test>upper]
#             plt.plot(anomalies, "o", markersize=10, label = "Anomalies")
    
#     error = mean_absolute_percentage_error(prediction, y_test)
#     plt.title("Mean absolute percentage error {0:.2f}%".format(error))
#     plt.legend(loc="best")
#     plt.tight_layout()
#     plt.grid(True);
    
# def plotCoefficients(model):
#     """
#         Plots sorted coefficient values of the model
#     """
    
#     coefs = pd.DataFrame(model.coef_, X_train.columns)
#     coefs.columns = ["coef"]
#     coefs["abs"] = coefs.coef.apply(np.abs)
#     coefs = coefs.sort_values(by="abs", ascending=False).drop(["abs"], axis=1)
    
#     plt.figure(figsize=(15, 7))
#     coefs.coef.plot(kind='bar')
#     plt.grid(True, axis='y')
#     plt.hlines(y=0, xmin=0, xmax=len(coefs), linestyles='dashed');
    

In [107]:
def smape_fast(y_true, y_pred):
    from numba import jit
    import math
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

from sklearn.preprocessing import StandardScaler as sc

In [53]:
def code_mean(data, cat_feature, real_feature):
    """
    Returns a dictionary where keys are unique categories of the cat_feature,
    and values are means over real_feature
    """
    return dict(data.groupby(cat_feature)[real_feature].mean())

def model_Validation (X, y,size, model):
    model = model

    # reserve 30% of data for testing
    X_train, X_valid, y_train, y_valid = timeseries_train_test_split(X, y, test_size=size)
    
    valid_time = X_valid.index
#     X_train = sc.fit_transform(X_train)
#     X_valid = sc.transform(X_valid)   
    model.fit(X_train, y_train)
    
    pred = model.predict(X_valid)
    error = smape_fast(pred, y_valid)
    print('SMAPE:',error)
    return X_train, X_valid, y_train, y_valid, valid_time, pred

In [44]:
data.head(1)

,y,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed,lag_24,lag_168
Time,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,0.431,8,29,1,10,0,1,0,0,0,1,0,19.3,0.0,5.0,NaN,NaN


In [60]:
_,_,_,y_valid,valid_time, pred_X303_LR = model_Validation(X,y, 0.3, LinearRegression())

SMAPE: 38.38744090546601


In [61]:
from sklearn.linear_model import LassoCV, RidgeCV

In [62]:
_,_,_,y_valid,valid_time, pred_X303_Ridge = model_Validation(X,y, 0.3, RidgeCV(cv=tscv))

SMAPE: 38.387631928105854


In [63]:
_,_,_,y_valid,valid_time,  pred_X303_Lasso = model_Validation(X,y, 0.3, LassoCV(cv=tscv))

SMAPE: 38.818674563873195


In [65]:
from xgboost import XGBRegressor 
xgb = XGBRegressor(verbosity=0)
_,_,_,y_valid,valid_time, pred_X303_XG = model_Validation(X,y, 0.3, xgb)

SMAPE: 39.00068030294938


In [ ]:
# ## 2018-4-18일부터

# xgb = XGBRegressor(verbose=True)
# _,_,_,y_valid,valid_time, pred_X303_XG = model_Validation(X_2,y_2, 0.3, xgb)

# df = pd.DataFrame(pred_X303_XG, index = valid_time)
# df['actual'] = y_valid
# df.plot(figsize = (30,30))

In [71]:
from lightgbm import LGBMRegressor
lgb = LGBMRegressor()
_,_,_,y_valid,valid_time, pred_X303_LGB = model_Validation(X,y, 0.3, lgb)

SMAPE: 38.16624034007779


In [67]:
####################
## LGBM 더미 없는 경우

dataT = pd.concat([pd.DataFrame(test.X303), TE_timeV_wthr], axis=1)
dataT = dataT.rename(columns={'X303':'y'})

for i in [24, 168]:
    dataT['lag_{}'.format(i)] = dataT.y.shift(i)

yT = dataT.dropna().y
XT = dataT.dropna().drop(['y'], axis=1)

In [68]:
from lightgbm import LGBMRegressor
lgb = LGBMRegressor()
_,_,_,y_valid,valid_time, pred_X303_LGB_T = model_Validation(XT,yT, 0.3, lgb)


SMAPE: 38.77066855638999


In [69]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
_,_,_,y_valid,valid_time, pred_X303_RF = model_Validation(X,y, 0.3, rf)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


SMAPE: 40.61606026229266


In [70]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor()
_,_,_,y_valid,valid_time, pred_X303_gb = model_Validation(X,y, 0.3, gb)


SMAPE: 38.95476427025396


In [ ]:
#     model_gb = GradientBoostingRegressor()
#     gb_param = {"loss":['ls', 'quantile'],
#         "learning_rate": [0.01,0.05, 0.1],
#         "max_depth": np.arange(3,15),
#         "n_estimators":[100, 1000],
#         "warm_start":[True, False]
#         }

#     grid_gb = GridSearchCV(model_gb, gb_param, cv=3)
#     grid_gb.fit(X_train, Y_trainLog)

In [72]:
# Make X_test데이터 셋
submission.columns

Index(['meter_id', 'X2018_7_1_1h', 'X2018_7_1_2h', 'X2018_7_1_3h',
       'X2018_7_1_4h', 'X2018_7_1_5h', 'X2018_7_1_6h', 'X2018_7_1_7h',
       'X2018_7_1_8h', 'X2018_7_1_9h', 'X2018_7_1_10h', 'X2018_7_1_11h',
       'X2018_7_1_12h', 'X2018_7_1_13h', 'X2018_7_1_14h', 'X2018_7_1_15h',
       'X2018_7_1_16h', 'X2018_7_1_17h', 'X2018_7_1_18h', 'X2018_7_1_19h',
       'X2018_7_1_20h', 'X2018_7_1_21h', 'X2018_7_1_22h', 'X2018_7_1_23h',
       'X2018_7_1_24h', 'X2018_7_1_d', 'X2018_7_2_d', 'X2018_7_3_d',
       'X2018_7_4_d', 'X2018_7_5_d', 'X2018_7_6_d', 'X2018_7_7_d',
       'X2018_7_8_d', 'X2018_7_9_d', 'X2018_7_10_d', 'X2018_7_m', 'X2018_8_m',
       'X2018_9_m', 'X2018_10_m', 'X2018_11_m'],
      dtype='object')

In [73]:
submission_hourly = pd.date_range(start = pd.datetime(2018, 7, 1, 0),
                             end = pd.datetime(2018, 7, 1, 23),
                             freq = "H")
submission_daily = pd.date_range(start = pd.datetime(2018, 7, 1),
                             end = pd.datetime(2018, 7, 10),
                             freq = "D")
# submission_monthly = pd.date_range(start = pd.datetime(2018, 7),
#                              end = pd.datetime(2018, 11),
#                              freq = "M")
submission_daily

DatetimeIndex(['2018-07-01', '2018-07-02', '2018-07-03', '2018-07-04',
               '2018-07-05', '2018-07-06', '2018-07-07', '2018-07-08',
               '2018-07-09', '2018-07-10'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
data.shape[0] % 168

In [ ]:
data.shape[0] - 110

In [ ]:
# 마지막 lag_168 은 110개가 채워진다. 
data.loc[:,['y','lag_168']][7224:]

In [ ]:
# 이전 168시간 데이터 확인 비교. 앞의 lag_168값과 일치한다. 중간의 110번쨰값(7166)은 위의 마지막 lag_!68값과 일치
# 그러면 111번째(7167)부터  24개 뽑아 대입
data.loc[:,['y','lag_168']][7056:7167]

In [ ]:
data.columns

In [74]:
y_testDaily = pd.DataFrame(index = submission_hourly)

tmp = dummies(dateTimeVar(y_testDaily), ['season', 'dayClass'])
tmp['season_1'] =0 ; tmp['season_3'] =0 ;tmp['season_4'] =0
tmp['dayClass_1'] =0 ; tmp['dayClass_2'] =0 
tmp['lag_24'] = y.tail(24).values
tmp['lag_168'] = data.loc[:,'lag_168'][7166:7190].values

X_test_H = pd.concat([tmp, w_hr2], axis=1,join='inner')

In [75]:
X_train_H = X
X_train_H.head(2)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed,lag_24,lag_168
Time,,,,,,,,,,,,,,,,
2017-09-05 10:00:00,9,5,1,10,0,0,1,0,0,1,0,22.5,0.0,2.5,0.242,0.431
2017-09-05 11:00:00,9,5,1,11,0,0,1,0,0,1,0,23.0,0.0,2.3,0.247,0.398


In [108]:
# X_train_H = sc.fit_transform(X_train_H)
# X_test_H =  sc.transform(X_train_H) 

TypeError: fit_transform() missing 1 required positional argument: 'X'

In [76]:
y_train_H = y

In [77]:
from sklearn.model_selection import GridSearchCV


## LGBReg, GridSearch trials

-1.17: boosting_type='goss', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=4, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=20000, subsample_freq=0
              
32: boosting_type='goss', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=50, n_jobs=4, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=20000, subsample_freq=0

37: boosting_type='dart', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=  50~100  , n_jobs=4, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=20000, subsample_freq=0

In [163]:
# import time
# start_time = time.time()
# print(start_time)
# lgb2 = LGBMRegressor(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
#               importance_type='split', learning_rate=0.15, max_depth=-1,
#               min_child_samples=30, min_child_weight=0.1, min_split_gain=0.0,
#               n_estimators=1000, n_jobs=4, num_leaves=31, objective=None,
#               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
#               subsample=1.0, subsample_for_bin=20000, subsample_freq=0, tree_learner='feature')

# _,_,_,y_valid,valid_time, pred_X303_lgb2 = model_Validation(XT,yT, 0.3, lgb2)

# print("--- %s seconds ---" % (time.time() - start_time))

1572091302.0948794
SMAPE: 39.46379689800117
--- 25.16241979598999 seconds ---


X303 최적: cv=3, error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=4, num_leaves=31,
                                     objective=None, random_state=None,
                                     reg_alpha=0.0, reg_lambda=...
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0, tree_learner='feature',
                                     verbose=1),
             iid='warn', n_jobs=4,

In [165]:
lg_param = {"boosting_type":['gbdt', 'dart','goss'],
    "learning_rate": [0.01,0.05,0.1,0.15],
    "min_child_samples":[10, 20, 30, 50],       
    "n_estimators":[50, 100, 200, 500],
    }

lgReg = LGBMRegressor(verbose=1,n_jobs=4, tree_learner='feature')
grid_gb = GridSearchCV(lgReg, lg_param, cv=3, n_jobs=4, verbose=10)


import time
start_time = time.time()
print(start_time)
grid_gb.fit(X_train_H, y_train_H)
print("--- %s seconds ---" % (time.time() - start_time))

1572091385.926902
Fitting 3 folds for each of 360 candidates, totalling 1080 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    7.9s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   15.7s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:   25.2s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   34.8s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   49.6s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:  2.4min
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed:  2.7min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  3.2min
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed:  3.6min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  4.1min
[Parallel(n_jobs=4)]: Done 173 tasks      | elapsed:  4.5min
[Parallel(

--- 1855.5285897254944 seconds ---


In [175]:
pred_H_X303 = grid_gb.predict(X_test_H)
pred_H_X303

array([0.60239669, 0.60239669, 0.60239669, 0.45234894, 0.60239669,
       0.63824301, 0.68578153, 0.60097243, 0.64094737, 0.80008631,
       0.82044701, 0.82314306, 0.82314306, 0.88642931, 0.91882165,
       0.91828641, 0.91828641, 0.91828641, 0.88117703, 0.8570603 ,
       0.8570603 , 0.87456723, 0.84991377, 0.8414207 ])

In [ ]:
agg = {}
for i in range(24):
    

# 칼럼 50개 돌리기

In [215]:
test50 = test.iloc[:,:50]
test50.head(1)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X371,X200,X111,X163,X364,X378,X271,X462,X427,X73
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.214,0.303754,0.445,0.203,0.249,0.22,0.311651,0.268794,0.27,0.008


In [218]:
lg_param = {"boosting_type":['gbdt', 'dart','goss'],
    "learning_rate": [0.01,0.05,0.1,0.15],
    "min_child_samples":[10, 20, 30, 50],       
    "n_estimators":[50, 100, 200, 500],
    }

def gsLGBR_Hrly_FitPred (model, params, mID, X_train, y_train, X_test):
    grid_gb = GridSearchCV(model, params, cv=3, n_jobs=4, verbose=10)
    grid_gb.fit(X_train, y_train)
    fcsrt = grid_gb.predict(X_test)
    a = pd.DataFrame([mID], columns=['meter_id'])
    for i in range(24):
        a['X2018_7_1_'+str(i+1)+'h']=[fcst[i]]
    return a
   
def test50_Hrly_GS_LGBReg(train, X_test_H):
    hrly_50 = pd.DataFrame
    for mID in train.columns:
        data = pd.concat([train.loc[:,[mID]], TE_timeV_dum_wthr], axis=1)
        data = data.rename(columns={mID:'y'})
        for i in [24, 168]:
            data['lag_{}'.format(i)] = data.y.shift(i)
        y_train = data.dropna().y
        X_train = data.dropna().drop(['y'], axis=1)
        
        tmp = gsLGBR_Hrly_FitPred(LGBMRegressor(verbose=1,n_jobs=4, tree_learner='feature'), lg_param,
                        mID, X_train, y_train, X_test_H)
        hrly_50 = pd.concat([hrly_50, tmp], axis=0)
    return hrly_50

In [219]:
pred_50 = test50_Hrly_GS_LGBReg(test50, X_test_H)

NameError: name 'X_test' is not defined

In [ ]:
import os
os.chdir(r'kaggle/working')
pred_50.to_csv(r'first50_Hrly.csv')

from IPython.display import FileLink
FileLink(r'first50_Hrly.csv')